# 1.1 OpenAI API Basics

This notebook covers the **foundational OpenAI APIs** for building AI applications:

1. **Chat Completions API**: The traditional API with manual conversation management
2. **Responses API**: The newer, stateful API (released March 2025)

Understanding both APIs will help you:
- Choose the right tool for your use case
- Understand how AI agents manage state
- Work with existing codebases using either API

<a target="_blank" href="https://githubtocolab.com/IT-HUSET/ai-agenter-2025/blob/main/exercises/openai/1.1-openai-api-basics.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup

In [ ]:
%pip install openai~=2.1 python-dotenv~=1.0 --upgrade --quiet

In [ ]:
import os
import json
from openai import OpenAI

# Check if running in Google Colab
try:
    from google.colab import userdata
    IN_COLAB = True
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Running in Google Colab - API key loaded from secrets")
except ImportError:
    IN_COLAB = False
    try:
        from dotenv import load_dotenv, find_dotenv
        load_dotenv(find_dotenv())
        print("✅ Running locally - API key loaded from .env file")
    except ImportError:
        print("⚠️ python-dotenv not installed")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found!")
    if IN_COLAB:
        print("   → Click the key icon (🔑) in the left sidebar and add 'OPENAI_API_KEY'")
else:
    print("✅ OpenAI client initialized")

---

## Part 1: Chat Completions API - The Foundation

The **Chat Completions API** is the traditional way to interact with OpenAI models.

**Key Concepts:**
- **Messages**: Conversation is a list of messages with roles
- **Roles**: `system`, `user`, `assistant`, `tool`
- **Manual State**: You manage conversation history yourself

**Why learn this?**
- Used in most existing codebases
- Fine-grained control over conversation
- Essential for understanding agent architecture

### Understanding Message Roles

In [ ]:
# Simple request with Chat Completions API
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "user", "content": "What is 2+2?"}
    ]
)

print("Response:")
print(response.choices[0].message.content)

### The Three Core Roles

1. **`system`**: Sets behavior and instructions (typically placed first)
2. **`user`**: User's input or questions
3. **`assistant`**: Model's responses (in history)

In [ ]:
# Using system message to set behavior
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant who always responds in rhyme."
        },
        {
            "role": "user",
            "content": "Tell me about Python programming"
        }
    ]
)

print("With system message:")
print(response.choices[0].message.content)

### Role Hierarchy in Chat Completions API

The Chat Completions API uses three primary roles:

**Role Priority & Purpose:**
- **`system`**: High-level, persistent behavior instructions (highest priority)
- **`user`**: User's input or questions
- **`assistant`**: Model's responses (in conversation history)

**Best Practice:**
```python
messages = [
    {"role": "system", "content": "You are a helpful coding assistant"},
    {"role": "user", "content": "How do I read a file in Python?"}
]
```

**Note:** The `system` role provides persistent instructions throughout the conversation. Unlike the Responses API (covered in Part 2), Chat Completions API uses `system`, `user`, `assistant`, and `tool` roles only.

In [ ]:
# Example: Using system role for persistent instructions
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {
            "role": "system",
            "content": """You are a helpful coding assistant. 
            When answering questions:
            1. Prefer concise answers with code examples
            2. Assume the user is using Python 3.12
            3. Include brief explanations"""
        },
        {
            "role": "user",
            "content": "How do I use match-case statements?"
        }
    ]
)

print("With detailed system instructions:")
print(response.choices[0].message.content)

In [ ]:
# Comparison: basic vs detailed system instructions
print("Basic system instruction:")
print("="*80)
response_basic = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain classes in Python"}
    ]
)
print(response_basic.choices[0].message.content)

print("\n\nDetailed system instruction:")
print("="*80)
response_detailed = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {
            "role": "system", 
            "content": """You are a helpful assistant. When explaining programming concepts:
            1. Use simple language (ELI15 level)
            2. Include a real-world analogy
            3. Provide one code example
            Keep total response under 100 words."""
        },
        {"role": "user", "content": "Explain classes in Python"}
    ]
)
print(response_detailed.choices[0].message.content)

print("\n\n💡 More detailed system instructions provide better control over output format and style!")

### Manual Conversation History Management

Unlike the Responses API, you must **manually track** the conversation history.

In [ ]:
# Start with a system message
messages = [
    {"role": "system", "content": "You are a helpful Python expert."}
]

# Turn 1: User asks a question
messages.append({
    "role": "user",
    "content": "How do I read a CSV file in Python?"
})

response_1 = client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages
)

# Add assistant's response to history
assistant_message_1 = response_1.choices[0].message.content
messages.append({
    "role": "assistant",
    "content": assistant_message_1
})

print("Turn 1:")
print(assistant_message_1)
print("\n" + "="*80 + "\n")

# Turn 2: Follow-up question (requires context)
messages.append({
    "role": "user",
    "content": "What if the file has headers?"
})

response_2 = client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages
)

assistant_message_2 = response_2.choices[0].message.content
print("Turn 2 (knows we're talking about CSV):")
print(assistant_message_2)

# Inspect the full message history
print("\n" + "="*80 + "\n")
print(f"Total messages in history: {len(messages)}")
for i, msg in enumerate(messages):
    print(f"{i+1}. {msg['role']}: {msg['content'][:50]}...")

### Conversation Manager Pattern

A common pattern is to create a helper class for managing conversation history.

In [ ]:
class ChatCompletionsConversation:
    """Helper class for managing Chat Completions API conversations"""
    
    def __init__(self, system_message: str = None, model: str = "gpt-5-mini"):
        self.model = model
        self.messages = []
        
        if system_message:
            self.messages.append({
                "role": "system",
                "content": system_message
            })
    
    def send(self, user_message: str, **kwargs) -> str:
        """Send a user message and get response"""
        # Add user message to history
        self.messages.append({
            "role": "user",
            "content": user_message
        })
        
        # Get completion
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            **kwargs
        )
        
        # Add assistant response to history
        assistant_message = response.choices[0].message.content
        self.messages.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message
    
    def get_history(self) -> list:
        """Get full conversation history"""
        return self.messages
    
    def reset(self):
        """Clear conversation history (keeps system message)"""
        system_messages = [msg for msg in self.messages if msg["role"] == "system"]
        self.messages = system_messages

# Test the conversation manager
conv = ChatCompletionsConversation(
    system_message="You are a concise coding assistant. Keep answers under 50 words."
)

print("Turn 1:")
print(conv.send("What's a list comprehension in Python?"))
print("\n" + "="*80 + "\n")

print("Turn 2:")
print(conv.send("Show me an example"))
print("\n" + "="*80 + "\n")

print(f"\nConversation has {len(conv.get_history())} messages")

### 🎯 Exercise 1: Build a Conversation Manager

**Task:** Enhance the `ChatCompletionsConversation` class:
1. Add a `token_limit` parameter
2. Implement automatic trimming when approaching the limit
3. Keep system message and most recent N messages
4. Test with a long conversation

In [ ]:
# YOUR CODE HERE

class SmartConversation(ChatCompletionsConversation):
    """Conversation manager with token limit handling"""
    
    def __init__(self, system_message: str = None, model: str = "gpt-5-mini", max_messages: int = 20):
        super().__init__(system_message, model)
        self.max_messages = max_messages
    
    def send(self, user_message: str, **kwargs) -> str:
        # TODO: Implement trimming logic before sending
        pass

# Test your implementation
# smart_conv = SmartConversation(max_messages=5)
# for i in range(10):
#     print(f"Turn {i+1}: {smart_conv.send(f'Tell me fact #{i+1} about Python')}")
# print(f"\nFinal message count: {len(smart_conv.get_history())}")

---

## Part 2: Responses API - Modern Approach

The **Responses API** (released March 2025) simplifies conversation management.

**Key Differences:**
- ✅ **Stateful**: Conversation history managed automatically
- ✅ **Simpler**: No need to track messages manually
- ✅ **Built-in tools**: Native web search, file search
- ✅ **Multimodal**: Easy image/audio support

**When to use which?**
- **Chat Completions**: Fine-grained control, existing codebases
- **Responses API**: New projects, simpler state management

### Basic Responses API Call

In [ ]:
# Simple request - much cleaner!
response = client.responses.create(
    model="gpt-5-mini",
    input="What is 2+2?"
)

print("Response:")
print(response.output_text)

### System Instructions with Responses API

In [ ]:
# System-level instructions (equivalent to system message)
response = client.responses.create(
    model="gpt-5-mini",
    instructions="You are a helpful assistant who always responds in rhyme.",
    input="Tell me about Python programming"
)

print("With instructions:")
print(response.output_text)

### Automatic Conversation History

The key advantage of Responses API is automatic state management via `previous_response_id`:

In [ ]:
# Turn 1: Ask a question
response_1 = client.responses.create(
    model="gpt-5-mini",
    input="How do I read a CSV file in Python?"
)

print("Turn 1:")
print(response_1.output_text)
print(f"\nResponse ID: {response_1.id}")
print("\n" + "="*80 + "\n")

# Turn 2: Follow-up (just reference previous response)
response_2 = client.responses.create(
    model="gpt-5-mini",
    input="What if the file has headers?",
    previous_response_id=response_1.id  # ✨ Magic! History managed for you
)

print("Turn 2 (automatically knows context):")
print(response_2.output_text)

### API Comparison: Key Differences

**Roles:**
- **Chat Completions**: `system`, `user`, `assistant`, `tool`
- **Responses API**: `developer`, `user`, `assistant` (+ `instructions` parameter)

**State Management:**
- **Chat Completions**: Manual (you manage message array)
- **Responses API**: Automatic (via `previous_response_id`)

**Instructions:**
- **Chat Completions**: `system` role message
- **Responses API**: `instructions` parameter OR `developer` role message

In [ ]:
print("CHAT COMPLETIONS API:")
print("="*80)
print("""
# Manual state management with system role
messages = [
    {"role": "system", "content": "You are helpful"},
    {"role": "user", "content": "Hello"}
]
response = client.chat.completions.create(model="gpt-5-mini", messages=messages)

# Manually append assistant response
messages.append({"role": "assistant", "content": response.choices[0].message.content})

# Manually append next user message
messages.append({"role": "user", "content": "Follow-up"})
response = client.chat.completions.create(model="gpt-5-mini", messages=messages)
""")

print("\nRESPONSES API:")
print("="*80)
print("""
# Automatic state management with instructions parameter
response_1 = client.responses.create(
    model="gpt-5-mini",
    instructions="You are helpful",  # or use developer role in input
    input="Hello"
)

# Just reference previous response - history managed automatically!
response_2 = client.responses.create(
    model="gpt-5-mini", 
    input="Follow-up", 
    previous_response_id=response_1.id
)
""")

print("\n✅ Responses API = Less boilerplate, automatic state management, developer/user role hierarchy")

### 🎯 Exercise 2: Convert to Responses API

**Task:** Take your `ChatCompletionsConversation` class and convert it to use the Responses API instead.

**Hints:**
1. Store `previous_response_id` instead of messages array
2. Use `instructions` parameter for system message
3. The code should be simpler!

In [ ]:
# YOUR CODE HERE

class ResponsesConversation:
    """Helper class for managing Responses API conversations"""
    
    def __init__(self, instructions: str = None, model: str = "gpt-5-mini"):
        self.model = model
        self.instructions = instructions
        self.previous_response_id = None
    
    def send(self, user_input: str, **kwargs) -> str:
        """Send a user message and get response"""
        # TODO: Implement using Responses API
        pass
    
    def reset(self):
        """Reset conversation"""
        self.previous_response_id = None

# Test your implementation
# conv = ResponsesConversation(instructions="You are a helpful assistant")
# print(conv.send("What's a decorator in Python?"))
# print("\n" + "="*80 + "\n")
# print(conv.send("Show me an example"))

---

## Summary

In this notebook, you learned:

✅ **Chat Completions API**: Manual history, role system, fine-grained control  
✅ **Responses API**: Automatic state management, simpler code  
✅ **When to use which**: Chat Completions for control, Responses for simplicity

**Key Takeaways:**
- Chat Completions requires manual message tracking (more control)
- Responses API handles state automatically (simpler)
- Both APIs can accomplish the same tasks, just with different patterns

**Next Steps:**
- **Notebook 1.2**: Advanced OpenAI features (function calling, reasoning modes, model selection)
- **Notebook 1.3**: Structured outputs with JSON schemas
- **Notebook 1.4**: Prompt engineering techniques

**Resources:**
- [Chat Completions API Docs](https://platform.openai.com/docs/api-reference/chat)
- [Responses API Docs](https://platform.openai.com/docs/api-reference/responses)